# 🟨 Métricas Derivadas (Gold) — Processos Regulatórios

Este notebook cria **métricas derivadas finais** (pensadas para BI) a partir dos dados consolidados (2018 + 2019+), seguindo as **regras de negócio** do projeto.

## ✅ Saídas geradas
- `output/fato_processo_regulatorio_com_metricas.csv` (FATO com métricas derivadas)
- `output/resumo_metricas.csv` (resumo rápido para QA)
- `output/dicionario_metricas.md` (mini-dicionário das métricas para colar no README/docs)

> **Observação**: este notebook não versiona a saída no GitHub quando ela for grande (boa prática).  
Data de referência (para métricas de “em aberto”): **2026-01-06**.


In [15]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, date


## 1) Leitura dos arquivos (2018 + 2019+)

O notebook tenta ler **primeiro na mesma pasta** (ex.: `gold/`), e se não encontrar, tenta `../silver/`.


In [16]:
BASE_DIR = Path().resolve()
OUT_DIR = BASE_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# tenta ler na pasta atual (ex.: gold/)
candidates_here = [
    BASE_DIR / "2018_anonimizado.xlsx",
    BASE_DIR / "2019_anonimizado.xlsx",
]

# fallback: ../silver/
candidates_silver = [
    BASE_DIR.parent / "gold" / "2018_anonimizado.xlsx",
    BASE_DIR.parent / "gold" / "2019_anonimizado.xlsx",
]

INPUT_FILES = [p for p in candidates_here if p.exists()]
if len(INPUT_FILES) == 0:
    INPUT_FILES = [p for p in candidates_silver if p.exists()]

print("📁 BASE_DIR:", BASE_DIR)
print("📥 INPUT_FILES encontrados:")
for p in INPUT_FILES:
    print(" -", p)

if len(INPUT_FILES) == 0:
    raise FileNotFoundError(
        "Não encontrei 2018_anonimizado.xlsx e 2019_anonimizado.xlsx nem na pasta atual nem em ../silver/."
    )

dfs = []
for f in INPUT_FILES:
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name  # opcional (debug)
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)
print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


📁 BASE_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold
📥 INPUT_FILES encontrados:
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2018_anonimizado.xlsx
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2019_anonimizado.xlsx
✅ Linhas/Colunas consolidadas: (732261, 87)


,ULTIMO_PROCESSO,SITUACAO_DO_PROCESSO,IS_SEDE_EAD,NO_DO_PROCESSO,MODALIDADE,ANO_DO_PROTOCOLO,DATA,ORGAO,ATO,CATEGORIA_ATO,...,CINE_AREA_ESPECIFICA,CODIGO_AREA_GERAL_CINE,AREA_GERAL_CINE,CODIGO_AREA_DETALHADA_CINE,AREA_DETALHADA_CINE,CODIGO_AREA_ESPECIFICA_CINE,AREA_ESPECIFICA_CINE,ROTULO_CINE,AVALIACAO_OFICIAL,fonte_arquivo
0,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
1,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
2,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
3,NÃO,Aguardando Pagamento,S,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
4,NÃO,Arquivado,N,20070028,PRESENCIAL,2008,2008-09-26 00:00:00,SERES/DIREG/CGRERCES,Reconhecimento de Curso,Curso,...,Humanidades (exceto línguas),2,Artes e humanidades,223,Filosofia e ética,22,Humanidades (exceto línguas),Filosofia,Regulação,2018_anonimizado.xlsx


## 2) Helpers de padronização (colunas e tipos)

Os dados podem estar com nomes de colunas em **maiúsculo + underscore** (ex.: `SITUACAO_DO_PROCESSO`) ou em **formato original** (ex.: `Situação do Processo`).  
Os helpers abaixo tentam encontrar automaticamente a coluna equivalente.


In [17]:
def pick_col(df: pd.DataFrame, *candidates: str):
    """Retorna o primeiro nome de coluna existente dentro de candidates, senão None."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def to_datetime_safe(s: pd.Series):
    return pd.to_datetime(s, errors="coerce", dayfirst=True)

def clean_str_series(s: pd.Series):
    return (
        s.astype(str)
         .str.strip()
         .replace({
             "": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan
         })
    )

def to_int_safe(s: pd.Series):
    return pd.to_numeric(s, errors="coerce").astype("Int64")


## 3) Campos-base (normalizações essenciais)

- UF (prioriza UF do processo; fallback para cadastro)
- Modalidade normalizada
- Pública vs Privada (Categoria Administrativa)
- Âmbito (Sistema de Ensino)
- Flags de divergência (endereço / vagas)
- Tempo de tramitação (dias) com correção de sinal

> **Obs.**: Ajuste as palavras-chave conforme a sua realidade de dados.


In [18]:
# -----------------------------
# Colunas (auto-pick)
# -----------------------------
col_uf_proc = pick_col(df, "UF_PROCESSO", "UF Processo")
col_uf_cad  = pick_col(df, "UF_CADASTRO", "UF Cadastro")

col_modal   = pick_col(df, "MODALIDADE", "Modalidade")
col_catadm  = pick_col(df, "CATEGORIA_ADMINISTRATIVA", "Categoria Administrativa")
col_sistens = pick_col(df, "SISTEMA_DE_ENSINO", "Sistema de Ensino")

col_end_div = pick_col(df, "ENDERECO_DIVERGENTE", "Endereço Divergente", "Endereco Divergente")
col_div_vag = pick_col(df, "TEM_DIVERGENCIA_VAGAS", "tem_divergencia_vagas")  # pode já existir
col_dif_vag = pick_col(df, "DIF_VAGAS_PROCESSO_CADASTRO", "dif_vagas_processo_cadastro")

col_vag_proc = pick_col(df, "VAGAS_SOLICITADAS_PROCESSO", "Vagas Solicitadas Processo")
col_vag_cad  = pick_col(df, "VAGAS_AUTORIZADAS_CADASTRO", "Vagas Autorizadas Cadastro")

col_data_proto = pick_col(df, "DATA", "Data")
col_data_ult_ato = pick_col(df, "DATA_DO_ULTIMO_ATO", "Data do Último Ato")
col_data_fase = pick_col(df, "DATA_DE_ENTRADA_FASE_ATUAL", "Data de Entrada Fase Atual")

col_tempo = pick_col(df, "tempo_tramitacao_dias", "TEMPO_TRAMITACAO_DIAS")

# -----------------------------
# UF consolidada
# -----------------------------
if col_uf_proc:
    df["UF"] = clean_str_series(df[col_uf_proc])
elif col_uf_cad:
    df["UF"] = clean_str_series(df[col_uf_cad])
else:
    df["UF"] = pd.NA

# -----------------------------
# Modalidade normalizada
# -----------------------------
if col_modal:
    m = clean_str_series(df[col_modal]).str.upper()
    df["Modalidade_norm"] = (
        m.replace({
            "EAD": "EAD",
            "À DISTÂNCIA": "EAD",
            "A DISTANCIA": "EAD",
            "PRESENCIAL": "PRESENCIAL",
            "SEMIPRESENCIAL": "SEMIPRESENCIAL",
            "HÍBRIDO": "SEMIPRESENCIAL",
            "HIBRIDO": "SEMIPRESENCIAL",
        })
    )
else:
    df["Modalidade_norm"] = pd.NA

# -----------------------------
# Pública vs Privada
# -----------------------------
if col_catadm:
    cat = clean_str_series(df[col_catadm]).str.upper()
    df["PublicaPrivada"] = np.where(cat.str.contains("PÚBLIC|PUBLIC", na=False), "PÚBLICA", "PRIVADA")
else:
    df["PublicaPrivada"] = pd.NA

# -----------------------------
# Âmbito administrativo (Sistema de Ensino)
# -----------------------------
if col_sistens:
    sist = clean_str_series(df[col_sistens]).str.upper()
    df["AmbitoAdministrativo"] = np.select(
        [
            sist.str.contains("FEDERAL", na=False),
            sist.str.contains("ESTADUAL", na=False),
            sist.str.contains("MUNICIPAL", na=False),
        ],
        ["FEDERAL", "ESTADUAL", "MUNICIPAL"],
        default="OUTROS"
    )
else:
    df["AmbitoAdministrativo"] = "DESCONHECIDO"

# -----------------------------
# Flags de divergência
# -----------------------------
# endereço divergente
if col_end_div:
    ed = clean_str_series(df[col_end_div]).str.upper()
    df["endereco_divergente_flag"] = ed.isin(["SIM", "TRUE", "1", "S"]).astype(int)
else:
    df["endereco_divergente_flag"] = 0

# divergência de vagas (se já existir, preserva; senão calcula)
if col_div_vag and col_div_vag in df.columns:
    df["tem_divergencia_vagas"] = to_int_safe(df[col_div_vag]).fillna(0).astype(int)
else:
    if col_vag_proc and col_vag_cad:
        vag_proc = pd.to_numeric(df[col_vag_proc], errors="coerce")
        vag_cad = pd.to_numeric(df[col_vag_cad], errors="coerce")
        df["dif_vagas_processo_cadastro"] = (vag_proc.fillna(0) - vag_cad.fillna(0))
        df["tem_divergencia_vagas"] = df["dif_vagas_processo_cadastro"].ne(0).astype(int)
    else:
        df["dif_vagas_processo_cadastro"] = np.nan
        df["tem_divergencia_vagas"] = 0

# -----------------------------
# Ano do Protocolo
# -----------------------------
col_ano_proto = pick_col(df, "ANO_DO_PROTOCOLO", "Ano do Protocolo")
df["AnoProtocolo"] = to_int_safe(df[col_ano_proto]) if col_ano_proto else pd.NA

# -----------------------------
# Tempo de tramitação (dias)
# prioridade:
# 1) se já existe coluna pronta
# 2) senão: Data_fase_atual - Data_protocolo (quando possível)
# 3) senão: Data_ultimo_ato - Data_protocolo
# -----------------------------
if col_tempo:
    df["tempo_tramitacao_dias"] = pd.to_numeric(df[col_tempo], errors="coerce")
else:
    df["tempo_tramitacao_dias"] = np.nan

# calcular se faltar
need_calc = df["tempo_tramitacao_dias"].isna().all()
if need_calc:
    d0 = to_datetime_safe(df[col_data_proto]) if col_data_proto else pd.NaT
    d1 = to_datetime_safe(df[col_data_fase]) if col_data_fase else pd.NaT
    d2 = to_datetime_safe(df[col_data_ult_ato]) if col_data_ult_ato else pd.NaT

    if col_data_proto and col_data_fase:
        df["tempo_tramitacao_dias"] = (d1 - d0).dt.days
    elif col_data_proto and col_data_ult_ato:
        df["tempo_tramitacao_dias"] = (d2 - d0).dt.days
    else:
        df["tempo_tramitacao_dias"] = np.nan

# limpar negativos (inversão de datas / problemas)
df.loc[df["tempo_tramitacao_dias"] < 0, "tempo_tramitacao_dias"] = np.nan

df[["AnoProtocolo","UF","Modalidade_norm","PublicaPrivada","AmbitoAdministrativo","tempo_tramitacao_dias","endereco_divergente_flag","tem_divergencia_vagas"]].head()


C:\Users\LeaoN\AppData\Local\Temp\ipykernel_5044\2534368523.py:9: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)


,AnoProtocolo,UF,Modalidade_norm,PublicaPrivada,AmbitoAdministrativo,tempo_tramitacao_dias,endereco_divergente_flag,tem_divergencia_vagas
0,2009,MG,EAD,PRIVADA,FEDERAL,NaN,0,0
1,2009,MG,EAD,PRIVADA,FEDERAL,NaN,1,0
2,2009,MG,EAD,PRIVADA,FEDERAL,NaN,1,0
3,2009,MG,EAD,PRIVADA,FEDERAL,NaN,0,0
4,2008,AC,PRESENCIAL,PRIVADA,FEDERAL,3646.0,0,1


## 4) Métricas derivadas (para BI)

### 4.1 Encerramento (proxy)
Baseado na **Situação do Processo** e na **Fase Atual** (quando presentes).

> Ajuste as listas de palavras-chave conforme o seu padrão interno.


In [19]:
# colunas de status/fase
col_situacao = pick_col(df, "SITUACAO_DO_PROCESSO", "Situação do Processo")
col_fase     = pick_col(df, "FASE_ATUAL", "Fase Atual")

situ = clean_str_series(df[col_situacao]).str.upper() if col_situacao else pd.Series([np.nan]*len(df))
fase = clean_str_series(df[col_fase]).str.upper() if col_fase else pd.Series([np.nan]*len(df))

# palavras-chave para encerramento administrativo
KW_ENCERRADO_SITU = ["CONCLU", "ENCERR", "ARQUIV", "FINALIZ"]
KW_ENCERRADO_FASE = [
    "PUBLICAÇÃO", "PUBLICACAO",
    "PORTARIA",
    "GABINETE DO MINISTRO",
    "DECISÃO FINAL", "DECISAO FINAL",
    "FINALIZAÇÃO NO SISTEMA", "FINALIZACAO NO SISTEMA",
    "ARQUIV"
]

# flags
df["proxy_situacao_concluida"] = situ.str.contains("|".join(KW_ENCERRADO_SITU), na=False)
df["proxy_fase_final"] = fase.str.contains("|".join(KW_ENCERRADO_FASE), na=False)

df["processo_encerrado"] = (df["proxy_situacao_concluida"] | df["proxy_fase_final"]).astype(int)

# tipo de encerramento (heurístico)
df["tipo_encerramento"] = np.select(
    [
        df["processo_encerrado"].eq(0),
        situ.str.contains("INDEFER", na=False),
        situ.str.contains("DEFER", na=False),
        situ.str.contains("ARQUIV", na=False) | fase.str.contains("ARQUIV", na=False),
    ],
    [
        "EM_ANDAMENTO",
        "INDEFERIDO",
        "DEFERIDO",
        "ARQUIVADO",
    ],
    default="ENCERRADO_ADMIN"
)

df[["processo_encerrado","tipo_encerramento", "proxy_situacao_concluida","proxy_fase_final"]].head(10)


,processo_encerrado,tipo_encerramento,proxy_situacao_concluida,proxy_fase_final
0,0,EM_ANDAMENTO,False,False
1,0,EM_ANDAMENTO,False,False
2,0,EM_ANDAMENTO,False,False
3,0,EM_ANDAMENTO,False,False
4,1,ARQUIVADO,True,True
5,1,ARQUIVADO,True,True
6,1,ARQUIVADO,True,False
7,1,ARQUIVADO,True,True
8,1,ARQUIVADO,True,False
9,1,ARQUIVADO,True,False


### 4.2 Ano de encerramento

Para processos encerrados, tentamos inferir o ano a partir de:
1) `DATA_DO_ULTIMO_ATO` (se existir)  
2) `DATA_DE_ENTRADA_FASE_ATUAL` (se existir)  
3) fallback: `AnoProtocolo` (último recurso, quando não há data fim)


In [20]:
d_ult = to_datetime_safe(df[col_data_ult_ato]) if col_data_ult_ato else pd.Series([pd.NaT]*len(df))
d_fase = to_datetime_safe(df[col_data_fase]) if col_data_fase else pd.Series([pd.NaT]*len(df))

ano_ult = d_ult.dt.year
ano_fase = d_fase.dt.year

df["ano_encerramento"] = np.where(
    df["processo_encerrado"].eq(1),
    ano_ult.fillna(ano_fase).fillna(df["AnoProtocolo"]).astype("Int64"),
    pd.NA
)

df[["processo_encerrado","ano_encerramento","AnoProtocolo"]].head(10)


,processo_encerrado,ano_encerramento,AnoProtocolo
0,0,<NA>,2009
1,0,<NA>,2009
2,0,<NA>,2009
3,0,<NA>,2009
4,1,2018.0,2008
5,1,2011.0,2007
6,1,2010.0,2007
7,1,2008.0,2007
8,1,2010.0,2007
9,1,2010.0,2007


### 4.3 Categorias de tempo e outliers

- `tempo_tramitacao_categoria`: Curto / Médio / Longo (bins simples e comunicáveis)
- `tempo_acima_mediana_global`: 1 se tempo > mediana global (para detectar desvios)
- `tempo_padronizado_zscore`: z-score (útil para ranking/outlier)


In [21]:
t = pd.to_numeric(df["tempo_tramitacao_dias"], errors="coerce")

# bins (ajuste se quiser)
bins = [0, 365, 730, np.inf]
labels = ["Curto (≤1 ano)", "Médio (1–2 anos)", "Longo (>2 anos)"]
df["tempo_tramitacao_categoria"] = pd.cut(t, bins=bins, labels=labels, right=True, include_lowest=True)

# mediana global (exclui nulos)
median_global = t.dropna().median() if t.notna().any() else np.nan
df["tempo_acima_mediana_global"] = np.where(t.notna() & (t > median_global), 1, 0)

# z-score
mu = t.dropna().mean() if t.notna().any() else np.nan
sd = t.dropna().std(ddof=0) if t.notna().any() else np.nan
df["tempo_padronizado_zscore"] = np.where(t.notna() & (sd is not np.nan) & (sd > 0), (t - mu) / sd, np.nan)

median_global, mu, sd


(np.float64(0.0),
 np.float64(338.44753321301994),
 np.float64(662.4846539691954))

### 4.4 Ato sensível e risco

Ato sensível (proxy) para: **Autorização / Credenciamento / Recredenciamento / Renovação / Reconhecimento**.

- `ato_sensivel_flag` (0/1)
- `flag_risco_alto` (0/1) — proxy
- `score_risco_regulatorio` (0–100) — pesos ajustáveis


In [22]:
col_ato = pick_col(df, "ATO", "Ato")
ato = clean_str_series(df[col_ato]).str.upper() if col_ato else pd.Series([np.nan]*len(df))

KW_ATO_SENSIVEL = ["AUTORIZ", "CREDENCI", "RECREDENCI", "RENOVA", "RECONHEC"]

df["ato_sensivel_flag"] = ato.str.contains("|".join(KW_ATO_SENSIVEL), na=False).astype(int)

# regras de risco (ajuste pesos/condições conforme governança)
cond_tempo_longo = df["tempo_tramitacao_categoria"].astype(str).str.contains("Longo", na=False)
cond_tempo_outlier = df["tempo_padronizado_zscore"].fillna(0) >= 2  # >=2 desvios padrão

df["flag_risco_alto"] = (
    (df["tem_divergencia_vagas"].fillna(0).astype(int) == 1) |
    (df["endereco_divergente_flag"].fillna(0).astype(int) == 1) |
    (df["ato_sensivel_flag"] == 1) |
    (cond_tempo_longo.fillna(False)) |
    (cond_tempo_outlier.fillna(False))
).astype(int)

# score (0–100) com pesos simples e interpretáveis
W_VAGAS = 30
W_END  = 20
W_TEMPO_LONGO = 25
W_ATO = 25

df["score_risco_regulatorio"] = (
    df["tem_divergencia_vagas"].fillna(0).astype(int) * W_VAGAS
    + df["endereco_divergente_flag"].fillna(0).astype(int) * W_END
    + cond_tempo_longo.fillna(False).astype(int) * W_TEMPO_LONGO
    + df["ato_sensivel_flag"].fillna(0).astype(int) * W_ATO
).clip(0, 100).astype(int)

df[["ato_sensivel_flag","flag_risco_alto","score_risco_regulatorio","tem_divergencia_vagas","endereco_divergente_flag","tempo_tramitacao_categoria"]].head(10)


,ato_sensivel_flag,flag_risco_alto,score_risco_regulatorio,tem_divergencia_vagas,endereco_divergente_flag,tempo_tramitacao_categoria
0,1,1,25,0,0,NaN
1,1,1,45,0,1,NaN
2,1,1,45,0,1,NaN
3,1,1,25,0,0,NaN
4,1,1,80,1,0,Longo (>2 anos)
5,1,1,80,1,0,Longo (>2 anos)
6,1,1,70,0,1,Longo (>2 anos)
7,1,1,55,1,0,Curto (≤1 ano)
8,1,1,50,0,0,Longo (>2 anos)
9,1,1,50,0,0,Longo (>2 anos)


### 4.5 Métricas de carga por entidade (IES / Curso / CINE)

- `qtd_processos_por_ies`
- `qtd_processos_por_curso`
- `qtd_processos_por_area_cine`

> Como os dados foram anonimizados, a chave típica de IES é `IES_ID_FAKE` (ou equivalente).


In [23]:
col_id_ies = pick_col(df, "IES_ID_FAKE", "CODIGO_DA_IES", "Código da IES")
col_id_curso = pick_col(df, "CODIGO_DO_CURSO", "Código do Curso", "CÓDIGO DO CURSO", "Codigo do Curso")

# CINE área geral (prioridade)
col_cine_geral = pick_col(df,
                         "AREA_GERAL_CINE", "CINE ÁREA GERAL", "CINE_AREA_GERAL",
                         "ROTULO_CINE", "ROTULO CINE")

# preencher "Não informado" na área geral se vazio
if col_cine_geral:
    df["cine_area_geral"] = clean_str_series(df[col_cine_geral]).fillna("Não informado")
else:
    df["cine_area_geral"] = "Não informado"

# IES
if col_id_ies:
    df["id_ies"] = clean_str_series(df[col_id_ies])
    df["qtd_processos_por_ies"] = df.groupby("id_ies")["id_ies"].transform("size")
else:
    df["id_ies"] = pd.NA
    df["qtd_processos_por_ies"] = np.nan

# Curso
if col_id_curso:
    df["id_curso"] = clean_str_series(df[col_id_curso])
    df["qtd_processos_por_curso"] = df.groupby("id_curso")["id_curso"].transform("size")
else:
    df["id_curso"] = pd.NA
    df["qtd_processos_por_curso"] = np.nan

# CINE geral
df["qtd_processos_por_area_cine"] = df.groupby("cine_area_geral")["cine_area_geral"].transform("size")

df[["id_ies","qtd_processos_por_ies","id_curso","qtd_processos_por_curso","cine_area_geral","qtd_processos_por_area_cine"]].head(10)


,id_ies,qtd_processos_por_ies,id_curso,qtd_processos_por_curso,cine_area_geral,qtd_processos_por_area_cine
0,3448,128,NaN,NaN,Não informado,220930
1,3448,128,NaN,NaN,Não informado,220930
2,3448,128,NaN,NaN,Não informado,220930
3,3448,128,NaN,NaN,Não informado,220930
4,3675,18,86920,6.0,Artes e humanidades,22890
5,1396,363,21489,10.0,Educação,127895
6,10086,2,NaN,NaN,Não informado,220930
7,1396,363,21489,10.0,Educação,127895
8,1675,191,NaN,NaN,Não informado,220930
9,1675,191,NaN,NaN,Não informado,220930


### 4.6 Gestão de acervo: tempo em aberto (para processos ativos)

- `processo_ativo` (0/1)
- `tempo_em_aberto_dias` (dias desde protocolo até hoje, para ativos)
- `faixa_tempo_em_aberto`

> A data de referência é a data do notebook. Para BI, você pode trocar por `TODAY()` em DAX, mas a métrica pré-calculada ajuda no QA.


In [24]:
from datetime import date

today = pd.Timestamp(date.today())

df["processo_ativo"] = (df["processo_encerrado"] == 0).astype(int)

if col_data_proto:
    d0 = to_datetime_safe(df[col_data_proto])

    df["tempo_em_aberto_dias"] = np.where(
        df["processo_ativo"] == 1,
        (today - d0).dt.days,
        np.nan
    )

    # limpar valores negativos (datas inconsistentes)
    df.loc[df["tempo_em_aberto_dias"] < 0, "tempo_em_aberto_dias"] = np.nan
else:
    df["tempo_em_aberto_dias"] = np.nan

# faixas simples
bins_open = [0, 365, 730, np.inf]
labels_open = ["Até 1 ano", "1–2 anos", "+2 anos"]

df["faixa_tempo_em_aberto"] = pd.cut(
    pd.to_numeric(df["tempo_em_aberto_dias"], errors="coerce"),
    bins=bins_open,
    labels=labels_open,
    include_lowest=True
)

df[["processo_ativo", "tempo_em_aberto_dias", "faixa_tempo_em_aberto"]].head(10)



C:\Users\LeaoN\AppData\Local\Temp\ipykernel_5044\2534368523.py:9: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)


,processo_ativo,tempo_em_aberto_dias,faixa_tempo_em_aberto
0,1,6159.0,+2 anos
1,1,6159.0,+2 anos
2,1,6159.0,+2 anos
3,1,6159.0,+2 anos
4,0,NaN,NaN
5,0,NaN,NaN
6,0,NaN,NaN
7,0,NaN,NaN
8,0,NaN,NaN
9,0,NaN,NaN


## 5) QA rápido (sanidade)

Gera um resumo para checar preenchimentos e distribuição das métricas.


In [25]:
resumo = pd.DataFrame({
    "linhas_total": [len(df)],
    "pct_encerrado": [round(df["processo_encerrado"].mean()*100, 2) if len(df) else np.nan],
    "pct_risco_alto": [round(df["flag_risco_alto"].mean()*100, 2) if len(df) else np.nan],
    "tempo_mediana": [pd.to_numeric(df["tempo_tramitacao_dias"], errors="coerce").median()],
    "tempo_media": [pd.to_numeric(df["tempo_tramitacao_dias"], errors="coerce").mean()],
    "pct_cine_nao_informado": [round((df["cine_area_geral"] == "Não informado").mean()*100, 2)],
})

display(resumo)

# salvar resumo QA
resumo.to_csv(OUT_DIR / "resumo_metricas.csv", index=False, encoding="utf-8-sig")
print("✅ Salvo:", OUT_DIR / "resumo_metricas.csv")


,linhas_total,pct_encerrado,pct_risco_alto,tempo_mediana,tempo_media,pct_cine_nao_informado
0,732261,84.27,77.2,0.0,338.447533,30.17


✅ Salvo: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\resumo_metricas.csv


## 6) Exportar FATO com métricas derivadas

Escolha um conjunto de colunas mínimo (FATO) + métricas derivadas.
Se preferir manter tudo, troque `df_fato = df[...]` por `df_fato = df.copy()`.


In [26]:
# tente achar identificador do processo (ou equivalente)
col_proc = pick_col(df, "NO_DO_PROCESSO", "Nº do Processo", "N_DO_PROCESSO", "NUMERO_DO_PROCESSO", "id_processo")
if col_proc:
    df["id_processo"] = clean_str_series(df[col_proc])
else:
    # fallback: cria id sintético
    df["id_processo"] = pd.Series(range(1, len(df)+1)).map(lambda x: f"PROC_{x:09d}")

cols_out = [
    "id_processo",
    "id_ies",
    "id_curso",
    "UF",
    "Modalidade_norm",
    "PublicaPrivada",
    "AmbitoAdministrativo",
    "AnoProtocolo",
    "cine_area_geral",
    "tempo_tramitacao_dias",
    "tempo_tramitacao_categoria",
    "tempo_acima_mediana_global",
    "tempo_padronizado_zscore",
    "processo_encerrado",
    "tipo_encerramento",
    "ano_encerramento",
    "processo_ativo",
    "tempo_em_aberto_dias",
    "faixa_tempo_em_aberto",
    "tem_divergencia_vagas",
    "endereco_divergente_flag",
    "ato_sensivel_flag",
    "flag_risco_alto",
    "score_risco_regulatorio",
    "qtd_processos_por_ies",
    "qtd_processos_por_curso",
    "qtd_processos_por_area_cine",
    "fonte_arquivo",
]

df_fato = df[[c for c in cols_out if c in df.columns]].copy()

out_path = OUT_DIR / "fato_processo_regulatorio_com_metricas.csv"
df_fato.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ Salvo:", out_path, "| linhas:", len(df_fato), "| colunas:", df_fato.shape[1])

df_fato.head()


✅ Salvo: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\fato_processo_regulatorio_com_metricas.csv | linhas: 732261 | colunas: 28


,id_processo,id_ies,id_curso,UF,Modalidade_norm,PublicaPrivada,AmbitoAdministrativo,AnoProtocolo,cine_area_geral,tempo_tramitacao_dias,...,faixa_tempo_em_aberto,tem_divergencia_vagas,endereco_divergente_flag,ato_sensivel_flag,flag_risco_alto,score_risco_regulatorio,qtd_processos_por_ies,qtd_processos_por_curso,qtd_processos_por_area_cine,fonte_arquivo
0,200810426,3448,NaN,MG,EAD,PRIVADA,FEDERAL,2009,Não informado,NaN,...,+2 anos,0,0,1,1,25,128,NaN,220930,2018_anonimizado.xlsx
1,200810426,3448,NaN,MG,EAD,PRIVADA,FEDERAL,2009,Não informado,NaN,...,+2 anos,0,1,1,1,45,128,NaN,220930,2018_anonimizado.xlsx
2,200810426,3448,NaN,MG,EAD,PRIVADA,FEDERAL,2009,Não informado,NaN,...,+2 anos,0,1,1,1,45,128,NaN,220930,2018_anonimizado.xlsx
3,200810426,3448,NaN,MG,EAD,PRIVADA,FEDERAL,2009,Não informado,NaN,...,+2 anos,0,0,1,1,25,128,NaN,220930,2018_anonimizado.xlsx
4,20070028,3675,86920,AC,PRESENCIAL,PRIVADA,FEDERAL,2008,Artes e humanidades,3646.0,...,NaN,1,0,1,1,80,18,6.0,22890,2018_anonimizado.xlsx


## 7) Mini-dicionário das métricas (para docs)

Gera um `dicionario_metricas.md` simples para você colar no README ou em `docs/`.


In [27]:
metricas_md = """# 📌 Mini-dicionário — Métricas Derivadas (Gold)

| Métrica | Tipo | Descrição (objetiva) |
|---|---:|---|
| processo_encerrado | 0/1 | Proxy de encerramento administrativo (situação/fase) |
| tipo_encerramento | texto | EM_ANDAMENTO / DEFERIDO / INDEFERIDO / ARQUIVADO / ENCERRADO_ADMIN |
| ano_encerramento | inteiro | Ano do encerramento (quando aplicável) |
| tempo_tramitacao_categoria | texto | Curto (≤1 ano) / Médio (1–2 anos) / Longo (>2 anos) |
| tempo_acima_mediana_global | 0/1 | 1 se tempo_tramitacao_dias > mediana global |
| tempo_padronizado_zscore | num | z-score do tempo de tramitação (outliers) |
| ato_sensivel_flag | 0/1 | 1 se ATO for sensível (autoriz./credenc./etc.) |
| flag_risco_alto | 0/1 | Proxy de risco alto (regras combinadas) |
| score_risco_regulatorio | 0–100 | Score ponderado (vagas/endereço/tempo/ato) |
| qtd_processos_por_ies | inteiro | Volume de processos associados à mesma IES |
| qtd_processos_por_curso | inteiro | Volume de processos associados ao mesmo curso |
| qtd_processos_por_area_cine | inteiro | Volume de processos por área CINE geral |
| processo_ativo | 0/1 | 1 se não encerrado |
| tempo_em_aberto_dias | inteiro | Dias desde o protocolo até hoje (somente ativos) |
| faixa_tempo_em_aberto | texto | Até 1 ano / 1–2 anos / +2 anos |
"""

md_path = OUT_DIR / "dicionario_metricas.md"
md_path.write_text(metricas_md, encoding="utf-8")
print("✅ Salvo:", md_path)


✅ Salvo: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\dicionario_metricas.md


# 📊 Alinhamento com Power BI (visuais recomendados)

Abaixo, um mapa simples: **métrica → visual → pergunta que responde**.

## 1) Visão Nacional (Página 1)
- **Card**: `COUNT(id_processo)` → “quantos processos no acervo?”
- **Card**: `% Encerrados` (média de `processo_encerrado`) → “qual parte já finalizou?”
- **Colunas**: `AnoProtocolo` × `COUNT(id_processo)` → evolução do volume
- **Mapa**: `UF` × `COUNT(id_processo)` → onde concentra
- **Barras**: `Modalidade_norm` × `%` (distribuição) → mudança EAD vs Presencial
- **Barras**: `cine_area_geral` × `COUNT(id_processo)` → carga por área

## 2) Performance / Prazos (Página 2)
- **Boxplot** (custom visual) ou **barras**: `UF` × `AVG(tempo_tramitacao_dias)`
- **Barras empilhadas**: `tempo_tramitacao_categoria` × `COUNT(id_processo)`
- **Tabela**: Top 20 processos por `tempo_tramitacao_dias` (outliers)

## 3) Risco e Priorização (Página 3)
- **Card**: `% Risco alto` (média `flag_risco_alto`)
- **Ranking**: `id_ies` × `score_risco_regulatorio` (média) + `qtd_processos_por_ies`
- **Scatter**: X=`qtd_processos_por_ies`, Y=`AVG(tempo_tramitacao_dias)`, size=`% risco alto`
- **Tabela**: processos ativos com `tempo_em_aberto_dias` alto

## 4) Ações sensíveis (Página 4)
- **Barras**: `ato` × `COUNT(id_processo)` e filtro `ato_sensivel_flag = 1`
- **Tabela**: processos sensíveis ativos (para gestão do fluxo)

> Dica: mantenha `score_risco_regulatorio` e `flag_risco_alto` como filtros (slicers).
